<a href="https://colab.research.google.com/github/rishabh135/2015/blob/master/Scraper_Race_1_Wallapop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What's the fastest scraper?

# Race 1: [Wallapop](https://wallapop.com)

Say you want to corner the used car market in Spain and you decide to track wallapop.

#### Which scraper should you use?

Here we'll explore [Puppeteer JS](https://pptr.dev/), [Playwright Python](https://playwright.dev/python/docs/intro) and [Selenium Python](https://selenium-python.readthedocs.io/index.html) as they all can handle the JS loaded content of this website.


### Scraping strategy

When scraping a website, I first identify the data source. If the webpage uses server-side rendering, it delivers data in HTML, making it easier to scrape with tools like Scrapy. However, this requires pinpointing relevant CSS classes. Alternatively, if the data comes via an API request, a more advanced scraper that can do the client side rendering is needed. In this race, we'll extract the data from the API responses by intercepting these.


_Results at the very bottom_

In [ ]:
# This is the URL we'll be scraping:
URL = 'https://es.wallapop.com/app/search?category_ids=100&filters_source=search_box&longitude=-3.69196&latitude=40.41956'

# ... and this is the API request path we'll be capturing:
API_URL = 'api/v3/cars/search'

## Puppeteer JS


### Preparation

Here we'll install NodeJS and Puppeteer, start a Node JS script and control its runtime with telekinesis.

In [ ]:
%%bash
# Most of this is to install NodeJS
sudo apt-get update
sudo apt-get install -y ca-certificates curl gnupg
sudo mkdir -p /etc/apt/keyrings
curl -fsSL https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg

NODE_MAJOR=18
echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_$NODE_MAJOR.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list

sudo apt-get update
sudo apt-get install nodejs -y

npm install telekinesis-js puppeteer
pip install telekinesis

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [962 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,254 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,059 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,230 kB]
Get:13 http://archive.ubuntu.com/ubun

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
npm notice 
npm notice New major version of npm available! 9.6.7 -> 10.1.0
npm notice Changelog: <https://github.com/npm/cli/releases/tag/v10.1.0>
npm notice Run `npm install -g npm@10.1.0` to update!
npm notice 


In [ ]:
import asyncio
import telekinesis as tk
import time

pods = []

broker = await tk.Broker().serve()
broker.entrypoint, _ = await tk.create_entrypoint(lambda pod: pods.append(pod))

In [ ]:
jsScript = """
const vm = require('vm');
const tk = require('telekinesis-js');
const process = require('process');

class Pod {
  start() {return new Promise(async resolve => {
    this.uncaughtExceptions = [];
    this.unhandledRejections = [];
    this.stop = resolve;
    await new tk.Entrypoint()(this)
    process.on('unhandledRejection', this.unhandledRejections.push)
    process.on('uncaughtException', this.uncaughtExceptions.push)
  })}
  async execute(code, inputs) {
    const prefix ='(async () => {' ;
    const suffix = '});'

    inputs = {...inputs} || {};
    inputs.require = require;
    inputs.setTimeout = setTimeout;
    let context = vm.createContext(inputs);
    const content = prefix + code + suffix;
    return await vm.runInContext(content, context)();
  }
}

new Pod().start()
"""
with open('script.js', 'w') as f:
  f.write(jsScript)


In [ ]:
p = await asyncio.create_subprocess_shell(
    'node script.js', stderr=asyncio.subprocess.PIPE, stdout=asyncio.subprocess.PIPE)

In [ ]:
# print(p.stderr.read())
puppeteerPod = pods[-1]

# Shameless plug: you can get a similar pod in 2 lines of code using PayPerRun, and it would cost you under 0.06/hr

# node = await tk.authenticate('wss://payper.run')
# puppeteerPod = await node.get('/>/market')().get('/>/compute/puppeteerjs')(cpus=0.8)

### Build Scraper

In [ ]:
browser = await puppeteerPod.execute("""
  const puppeteer = require('puppeteer');
  return await puppeteer.launch({args: ['--no-sandbox'], headless: true});
""")

In [ ]:
page = await browser.newPage()
await page.goto(URL)


≈ [object Object]

In [ ]:
print((await page.evaluate('document.body.innerText'))[:200])

Filtros
Coches
España, Madrid
Precio
Sólo profesionales
Marca y modelo
Año
Km
Ordenar por:
Relevancia

Copyright © 2023 Wallapop © de sus respectivos propietarios

Wallapop
Quiénes somos
Cómo funciona


In [ ]:
# Sometimes, I also create a short script on my browser that fetches some data and then I compare results

await page.evaluate("Array.from(document.querySelectorAll('.ItemCardWide__title')).map(x => x.innerText).slice(0, 10)")

['Audi Q5 S line 40 TDI quattro 140 kW (190 CV) S tronic',
 'Audi A3 Sportback S Line edition 2.0 TFSI quattro 140 kW (190 CV) S tronic',
 'Volkswagen Beetle Cabrio Design 1.4 TSI 110 kW (150 CV) DSG',
 'Audi Q7 3.0 TDI e-tron quattro 275 kW (373 CV) tiptronic',
 'Ford C-Max 1.0 EcoBoost Titanium 92 kW (125 CV)',
 'Volkswagen Up Move up! 1.0 44 kW (60 CV)',
 'MINI MINI 5 Puertas Cooper 100 kW (136 CV)',
 'Fiat 500X 1.6 MultiJet Lounge 4x2 88 kW (120 CV)',
 'Volkswagen T-Roc Advance 1.6 TDI 85 kW (115 CV)',
 'Volkswagen Tiguan Sport 2.0 TDI BMT 4Motion 140 kW (190 CV) DSG']

so far, so good...

In this benchmark, we'll intercept API data for clean JSON output. Specifically, a network request to /api/v3/cars/search provides the data. This can be found in the browser's Developer Tools under the Network tab, filtered by XHR and sorted by size.

While reverse engineering the API could save scraper time, we'll navigate the site and collect responses. Using Puppeteer, we'll set up event listeners for responses from specific URLs, effectively creating a response interceptor.

Let's do this!

In [ ]:
# telekinesis provides a _register_method that makes it easier to push code to pods like the one we're using
puppeteerPod._register_magic(get_ipython())

In [ ]:
%%puppeteerPod .execute
const puppeteer = require('puppeteer');
class Interceptor {
  async start(apiUrl) {
    this.browser = await puppeteer.launch({args: ['--no-sandbox'], headless: true});
    this.page = await this.browser.newPage()
    this.responses = [];
    this.page.on("response", async (r) => {
      if (r.url().includes(apiUrl)) {
        this.responses.push(await r.json());
      }
    });
    return this;
  }
}
return new Interceptor();

In [ ]:
# the ._last_magic captures the output of the cell magic we just performed
interceptor = await puppeteerPod._last_magic
await interceptor.start(API_URL)

≈ [object Object]

In [ ]:
# Now let's try this interceptor out...

await interceptor.page.goto(URL)

≈ [object Object]

In [ ]:
# await interceptor.responses # we can see the responses, but each response is a pretty big dict to print nicely here
await interceptor.responses.length

0

Wallapop is not paginated, instead, it uses an infinite scroll that is activated by a "ver más productos" button

In [ ]:
# Let's click this button!

await interceptor.page.evaluate('document.querySelector("#btn-load-more").click()')

In [ ]:
# Now lets see if we have more responses (we had only 1 before)

await interceptor.responses.length

0

In [ ]:
# Now let's scroll to the bottom, see if we get more responses

clientHeight = await interceptor.page.evaluate('document.body.clientHeight')
print(clientHeight)
await interceptor.page.mouse.wheel({'deltaY': clientHeight})

18049


In [ ]:
# Now let's see if we have more responses (we can repeat the last two steps a few times to check it keeps working)

await interceptor.responses.length

3

For this scrape, we have all the essentials. Time to construct a scraper that loads a specified number of items!

### Race

In [ ]:
%%puppeteerPod .execute
const puppeteer = require('puppeteer');

class WallapopScraper {
  async prepare(apiUrl) {
    this.browser = await puppeteer.launch({args: ['--no-sandbox'], headless: true});
    this.page = await this.browser.newPage();
    this.data = [];
    this.errors = [];
    this.page.on("response", async (r) => {
      if (r.url().includes(apiUrl)) {
        try {
          this.data.push(...(await r.json()).search_objects);
        } catch (e) {
          this.errors.push(e.message);
        }
      }
    });
    return this;
  }
  async start(url, items=200) {
    await this.page.goto(url);

    while (true) {
      if (this.data.length >= items) {
        return this.data.length;
      }
      let clientHeight = await this.page.evaluate('document.body.clientHeight')
      await this.page.mouse.wheel({deltaY: clientHeight})
      await this.page.evaluate('document.querySelector("#btn-load-more")?.click()')
      await new Promise(r => setTimeout(r, 500));
    }
  }
}
return new WallapopScraper();

In [ ]:
scraper = await puppeteerPod._last_magic
await scraper.prepare(API_URL)

≈ [object Object]

In [ ]:
t0 = time.time()

task = asyncio.create_task(scraper.start(URL, 2000)._execute())

while True:
  done, pending = await asyncio.wait({task}, timeout=5, return_when=asyncio.FIRST_COMPLETED)
  if task in done:
    print(time.time()-t0, task.result())
    break
  else:
    print(time.time()-t0, await scraper.data.length)

5.004951000213623 0
10.078546524047852 80
15.146006107330322 160
20.27302122116089 320
25.328995943069458 440
30.366957902908325 560
35.4189248085022 720
40.47896361351013 880
45.52199149131775 1000
50.60562586784363 1080
55.66694164276123 1160
60.721099615097046 1280
65.78897905349731 1360
70.86098313331604 1440
75.93797755241394 1520
81.01298332214355 1600
86.05596733093262 1680
91.1039776802063 1760
96.17195439338684 1840
101.21795463562012 1960
106.2649438381195 2000
106.92452836036682 2000


Took 107 seconds to scrape the first 2000 Wallapop posts using Puppeteer in Google Colab.

## Playwright - Python

One good thing about Playwright and Puppeteer is that they share a lot of their API, making it very easy to convert one into the other.

Another great thing about playwright is that it takes only two lines to get started

In [ ]:
!pip install playwright
!playwright install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 34.4 MB/s eta 0:00:00
148.1 Mb [] 0% 0.0s148.1 Mb [] 0% 28.6s148.1 Mb [] 0% 17.2s148.1 Mb [] 0% 8.1s148.1 Mb [] 1% 4.7s148.1 Mb [] 2% 3.6s148.1 Mb [] 2% 3.8s148.1 Mb [] 3% 3.4s148.1 Mb [] 4% 3.2s148.1 Mb [] 5% 2.9s148.1 Mb [] 7% 2.4s148.1 Mb [] 8% 2.2s148.1 Mb [] 9% 2.0s148.1 Mb [] 10% 2.1s148.1 Mb [] 11% 2.0s148.1 Mb [] 12% 1.9s148.1 Mb [] 13% 1.8s148.1 Mb [] 14% 1.8s148.1 Mb [] 15% 1.7s148.1 Mb [] 16% 1.7s148.1 Mb [] 18% 1.6s148.1 Mb [] 19% 1.5s148.1 Mb [] 20% 1.5s148.1 Mb [] 21% 1.5s148.1 Mb [] 23% 1.4s148.1 Mb [] 24% 1.4s148.1 Mb [] 25% 1.4s148.1 Mb [] 27% 1.3s148.1 Mb [] 28% 1.3s148.1 Mb [] 30% 1.2s148.1 Mb [] 31% 1.2s148.1 Mb [] 32% 1.2s148.1 Mb [] 33% 1.1s148.1 Mb [] 35% 1.1s148.1 Mb [] 36% 1.1s148.1 Mb [] 37% 1.1s148.1 Mb [] 38% 1.0s148.1 Mb [] 39% 1.0s148.1 Mb [] 41% 1.0s148.1 Mb [] 42% 1.0s148.1 Mb [] 44% 0.9s148.1 Mb [] 45% 0.9s148.1 Mb [] 46% 0.9s148.1 Mb [] 47% 0.9s148.1 Mb [] 48% 0.8s148.1 Mb [] 50% 0.8s148.1 Mb [

In [ ]:
from playwright.async_api import async_playwright

class WallapopScraper:
  def __init__(self):
    self.data = []
    self.tasks = []

  async def prepare(self, api_url):
    self.api_url = api_url
    async with async_playwright() as playwright:
      browser = await playwright.chromium.launch()
      self.page = await browser.new_page()
      return self

  async def start(self, url, items=200):
    async with async_playwright() as playwright:
      browser = await playwright.chromium.launch()
      self.page = await browser.new_page()

      self.page.on('response', self._handle_response_task)

      await self.page.goto(url)

      while True:
        if self.data_length >= items:
          return self.data_length

        client_height = await self.page.evaluate('document.body.clientHeight')
        await self.page.mouse.wheel(0, client_height)
        await self.page.evaluate('document.querySelector("#btn-load-more")?.click()')
        await self.page.wait_for_timeout(500)

  def _handle_response_task(self, response):
    if self.api_url in response.url:
      self.tasks.append(asyncio.create_task(self._handle_response(response)))

  async def _handle_response(self, response):
    self.data.extend((await response.json())['search_objects'])

  @property
  def data_length(self):
    return len(self.data)

scraper = await WallapopScraper().prepare(API_URL)

In [ ]:
t0 = time.time()

task = asyncio.create_task(scraper.start(URL, 2000))

while True:
  done, pending = await asyncio.wait({task}, timeout=5, return_when=asyncio.FIRST_COMPLETED)
  if task in done:
    print(time.time()-t0, task.result())
    break
  else:
    print(time.time()-t0, scraper.data_length) # Small change here data.length -> data_length

5.002890110015869 0
10.003755331039429 0
15.004281044006348 80
20.00675082206726 160
25.00812864303589 280
30.01045870780945 400
35.011797189712524 520
40.014413595199585 600
45.02244019508362 720
50.024412870407104 800
55.02506422996521 920
60.02603006362915 1000
65.02709293365479 1080
70.02847170829773 1160
75.02938604354858 1200
80.03241968154907 1280
85.03357410430908 1360
90.03441548347473 1400
95.03514885902405 1480
100.04125618934631 1520
105.04241251945496 1600
110.07725548744202 1640
115.08814287185669 1720
120.09092426300049 1760
125.09193539619446 1800
130.0953938961029 1840
135.09872007369995 1880
140.10433650016785 1920
145.10545134544373 1960
148.48783564567566 2000


Guess it took quite a bit longer, 148s is 38% over the Puppeteer JS one! Let's look at Selenium now

## Selenium (Python)

To intercept the network requests with selenium, we need a proxy. Luckly the 'selenium_wire' library takes care of it, making this pretty easy. Sure, a bit more cumbersome than the other tools, but no big deal.

### Preparation

To use selenium we need to install chrome and the chrome driver first

In [ ]:
%%shell
apt update
apt install -y wget git curl gnupg unzip libgconf-2-4
wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list'
apt-get -y update
apt-get install -y google-chrome-stable
wget -O /tmp/chromedriver-linux64.zip https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/`curl -sS https://googlechromelabs.github.io/chrome-for-testing/LATEST_RELEASE_STABLE`/linux64/chromedriver-linux64.zip

unzip /tmp/chromedriver-linux64.zip chromedriver-linux64/chromedriver -d /usr/local/bin/

pip install selenium selenium_wire

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://deb.nodesource.com/node_18.x nodistro InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 110 kB in 4s (29.2 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
16 packages can be upgraded. Run 'apt list --upgradable' to see

### Build Scraper

In [ ]:
from seleniumwire import webdriver    # We replace selenium with seleniumwire

options = webdriver.ChromeOptions() # If you don't use selenium wire, this would be webdriver.chrome.options.Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

In [ ]:
# Let's intercept those nice API requests

driver.scopes = [
  API_URL
]

In [ ]:
# selenium uses 'get' instead of 'goto'

driver.get(URL)

In [ ]:
# After a couple seconds we should see a request captured:

len(driver.requests)

1

In [ ]:
# So far so good... now let's decode the response (which is a little more involved here)

from seleniumwire.utils import decode
import json

response = driver.requests[-1].response

results = json.loads(decode(response.body, response.headers.get('Content-Encoding', 'identity')).decode())

In [ ]:
results.keys()

dict_keys(['search_objects', 'from', 'to', 'distance_ordered', 'search_point'])

In [ ]:
# let's spot-check the objects returned

results['search_objects'][30]

{'id': 'v6g2lw1xe7ze',
 'type': 'cars_search_cars',
 'content': {'id': 'v6g2lw1xe7ze',
  'title': 'Renault Clio Zen E-Tech Híbrido 103 kW (140 CV)',
  'storytelling': '',
  'distance': 0.0,
  'images': [{'original': 'https://cdn.wallapop.com/images/10420/fd/if/__/c10420p929687840/i3446476992.jpg?pictureSize=W800',
    'xsmall': 'https://cdn.wallapop.com/images/10420/fd/if/__/c10420p929687840/i3446476992.jpg?pictureSize=W320',
    'small': 'https://cdn.wallapop.com/images/10420/fd/if/__/c10420p929687840/i3446476992.jpg?pictureSize=W320',
    'large': 'https://cdn.wallapop.com/images/10420/fd/if/__/c10420p929687840/i3446476992.jpg?pictureSize=W800',
    'medium': 'https://cdn.wallapop.com/images/10420/fd/if/__/c10420p929687840/i3446476992.jpg?pictureSize=W640',
    'xlarge': 'https://cdn.wallapop.com/images/10420/fd/if/__/c10420p929687840/i3446476992.jpg?pictureSize=W800',
    'original_width': 0,
    'original_height': 0},
   {'original': 'https://cdn.wallapop.com/images/10420/fd/if/__/

In [ ]:
# All we have to do now is scroll down and click the 'ver más productos' button

driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
driver.execute_script('document.querySelector("#btn-load-more")?.click()')

In [ ]:
# Let's check if we intercepted anything new (it was 1 before)

len(driver.requests)

2

We have all we need... ready to build the scraper!

### Race

In [ ]:
class WallapopScraper:
  def __init__(self, api_url):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    self.driver = webdriver.Chrome(options=options)
    self.driver.scopes = [api_url]
    self.data = []
    self.driver.response_interceptor = self._handle_response

  async def start(self, search_url, items=200):
    self.driver.get(search_url)

    while len(self.data) < items:
      self.driver.execute_script('window.scrollTo(0, document?.body?.scrollHeight || 0);')
      self.driver.execute_script('document.querySelector("#btn-load-more")?.click()')

      await asyncio.sleep(0.5)
    return self.data_length

  def _handle_response(self, _, response):
    new_items = self._decode(response)['search_objects']
    self.data.extend(new_items)

  def _decode(self, response):
    return json.loads(decode(response.body, response.headers.get('Content-Encoding', 'identity')).decode())

  @property
  def data_length(self):
    return len(self.data)

scraper = WallapopScraper(API_URL)

In [ ]:
t0 = time.time()

task = asyncio.create_task(scraper.start(URL, 2000))

while True:
  done, pending = await asyncio.wait({task}, timeout=5, return_when=asyncio.FIRST_COMPLETED)
  if task in done:
    print(time.time()-t0, task.result())
    break
  else:
    print(time.time()-t0, scraper.data_length) # Small change here data.length -> data_length

9.893239259719849 0
18.618242502212524 40
23.619502782821655 80
28.6215078830719 160
35.91853046417236 280
41.50566339492798 320
47.298896074295044 400
52.72755742073059 480
62.26958131790161 600
70.16686534881592 680
80.56938314437866 800
86.38859820365906 840
94.91015911102295 960
105.94160962104797 1080
113.23581528663635 1160
120.86421179771423 1240
128.92929673194885 1320
133.9306058883667 1360
141.40202164649963 1440
146.40575623512268 1520
153.81019258499146 1560
161.5185511112213 1640
169.7215597629547 1720
179.22710132598877 1800
184.71230292320251 1840
189.71377992630005 1880
199.10692429542542 2000
199.6129014492035 2000


Looks like selenium took a bit longer than the other two: 200 seconds. Let's discuss the results

# Results

# Results

1. **Puppeteer JS:** 107s
2. **Playwright Python:** 148s
3. **Selenium Python:** 200s

Puppeteer leads, Playwright follows, and Selenium took almost double.

A couple things to note though:
- This result is particular to this webpage and scraping method [1]. As a benchmark, this isn't scientific and your milage will vary. But, it was a fun experiment and it might be useful as a template to try the different tools on another website.
- I ran this race a couple times here on Colab and I got different times (always the same ranking, even when running the scrapers in reverse order!). I believe it may be because Colab throttles the resources on the free plan, but there's no way to check. When I ran this on PayPerRun, where the resources allocated are more consistent, [I got more consistent times](https://payperrun.com/e-neuman/fun/scraper%2520race/1.%2520wallapop/).

In any case, if we look at cost, even Selenium amounted to just ~$0.0025 (if we ran it on PayPerRun) for the 2000 posts. We would have to scrape tens of millions of posts to even come close to the engineering/development cost. So that's likely were you want to optimize: do more in less of **your** time!

If you like this simple format of just scraping a website and comparing speed, drop me a comment/vote on reddit. Feedback is welcome! Open to scraping other websites and testing different methods.

_[1] I also tested browsing the website without capturing and decoding the API requests to explore why Selenium was slower, [check it out on PayPerRun!](https://payperrun.com/e-neuman/fun/scraper%20race/1.%20wallapop?display=test%20-%20no%20request%20intercepting)._